# notebook n04: Validation of Freezing

Jose Oliveira da Cruz, PhD  | LeDoux Lab  
jose.cruz@nyu.edu  

<img src="LedouxLab_logo.jpg" style="width: 300.464px; height: 100px; margin: 0px;">   


This notebooks takes the .csv output from nb03 and computes some validation statistics for the freezing algorithm.

### Import dependencies

In [ ]:
# Standart Library
import sys
import os

# Data Analysis
import numpy as np
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical Analysis
import pingouin as pg
from scipy import stats
import sklearn

# To Add my code to the 
sys.path.append(r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\src')

<a id='step_1'><a/>
# 1) Load the data: `_merged_summary_stats.csv`

In [ ]:
#where is the file
dir_path = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\processed\EXP004\individual_summary_stats'
file_path = r'jc_exp004_20200110_tes01_r_merged_summary_stats.csv'

summary_stats_dlc = pd.read_csv(os.path.join(dir_path, file_path), index_col=0)

In [ ]:
summary_stats_dlc.head()

# 2) Create a Validation DataSet


### 2.1) Prepare data from dlc to analyse

In [ ]:
# Subset by condition
condition = summary_stats_dlc['cs_epoch']=='peri_cs'

# Select only certain columns
summary_stats_dlc = summary_stats_dlc[condition][['animal_id', 'cs_id', 'cs_epoch', 'freezing_norm']]

In [ ]:
summary_stats_dlc.rename(columns={'freezing_norm': 'freezing_norm_dlc'}, inplace=True)

In [ ]:
summary_stats_dlc.sort_values(by=['animal_id', 'cs_id'], inplace=True)

In [ ]:
summary_stats_dlc

### 2.2) Prepare data from rodrigo to analyse

In [ ]:
#where is the file
dir_path_rt = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\external\RT\EXP004'
file_path_rt = r'RT_EXP004_norm_data_2nd_scoring.csv'

summary_stats_rt = pd.read_csv(os.path.join(dir_path_rt, file_path_rt), index_col=0
                              )
summary_stats_rt = summary_stats_rt.drop(columns=['Group', 'Sex', 'Exp', 'Session'])

In [ ]:
# put columns names in lowercase
summary_stats_rt.columns = summary_stats_rt.columns.str.lower()
# Add new columns with the cs_epoch
summary_stats_rt['cs_epoch'] = 'peri_cs'

In [ ]:
# Reorder the columns
summary_stats_rt = summary_stats_rt[['animal_id', 'cs_epoch']+[f'cs_0{n}' for n in range(1, 6)]]

In [ ]:
# Transform the table into the same format as the dlc
summary_stats_rt = summary_stats_rt.melt(
    id_vars=['animal_id', 'cs_epoch'],
    value_vars=[f'cs_0{n}' for n in range(1, 6)],
    value_name='freezing_norm_rt',
    var_name='cs_id',
)

In [ ]:
# Correct normalization (0 to 1, instead of 0 to 100)
summary_stats_rt['freezing_norm_rt'] = summary_stats_rt['freezing_norm_rt'].agg(lambda x: round(x/100, 2))

In [ ]:
summary_stats_rt.sort_values(by=['animal_id', 'cs_id'], inplace=True)

In [ ]:
summary_stats_rt.describe()

### 2.3 Prepare data from eztrack to analyse

In [ ]:
dir_path_ez = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\external\EZ\EXP004'
file_path_ez = r'JC_EXP004_20200110_TES01_R_SummaryStats_EZ.xls'

summary_stats_ez = pd.read_excel(
    os.path.join(dir_path_ez, file_path_ez),
    index_col=0,
)

summary_stats_ez.head()

In [ ]:
# Drop Unnecessary columns
summary_stats_ez.drop(columns=['Exp', 'Sex', 'Group', 'Session'], inplace=True)

In [ ]:
# Lowercase title for columns
summary_stats_ez.columns = summary_stats_ez.columns.str.lower()

In [ ]:
summary_stats_ez['cs_epoch'] = 'peri_cs'

In [ ]:
# Transform the table into the wide format
summary_stats_ez = summary_stats_ez.melt(
    id_vars=['animal_id', 'cs_epoch'],
    value_vars=[f'cs_0{n}' for n in range(1, 6)],
    var_name='cs_id',
    value_name='freezing_norm_ez',
)

In [ ]:
# Correct normalization (0 to 1, instead of 0 to 100)
summary_stats_ez['freezing_norm_ez'] = summary_stats_ez['freezing_norm_ez'].agg(lambda x: round(x/100, 2))

In [ ]:
summary_stats_ez.sort_values(by=['animal_id', 'cs_id'], inplace=True)

### 2.4) Merge all dataframes

In [ ]:
print(f'shape of the dataframes {summary_stats_dlc.shape}, {summary_stats_rt.shape}, {summary_stats_ez.shape}')

In [ ]:
main = pd.concat(
    [summary_stats_dlc.reset_index(drop=True),
     summary_stats_ez.reset_index(drop=True),
     summary_stats_rt.reset_index(drop=True),
    ],
    axis=1,
)

In [ ]:
main = main.loc[:, ~main.columns.duplicated()]
main.head()

In [ ]:
# Data read for analysis
summary_stats_main = main.copy()

# 3) Data Visualization and Analysis

### 3.1) Correlations and Regressions

In [ ]:
sns.set(
    context='talk',
    style='darkgrid',
    palette='colorblind',
    font='sans-serif',
    font_scale=1,
    color_codes=True,
    rc=None,
)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.regplot(
    x='freezing_norm_rt',
    y='freezing_norm_ez',
    data=summary_stats_main,
    ax=axes[0]
)

sns.regplot(
    x='freezing_norm_rt',
    y='freezing_norm_dlc',
    data=summary_stats_main,
    ax=axes[1]
)
for n in range(2):
    axes[n].set(ylim=(0, 1), xlim=(0.,1))
fig.suptitle('Linear Regression Plot: \n Left: Rodrigo vs EZ-Track | Right: Rodrigo vs DeepLabCut', y=1.05)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.residplot(
    x='freezing_norm_rt',
    y='freezing_norm_ez',
    data=summary_stats_main,
    ax=axes[0]
)

sns.residplot(
    x='freezing_norm_rt',
    y='freezing_norm_dlc',
    data=summary_stats_main,
    ax=axes[1]
)

fig.suptitle('Residual Plot for Linear Regression Fit: \n Left: Rodrigo vs EZ-Track | Right: Rodrigo vs DeepLabCut', y=1.05)


### 3.2) Pearson's Correlation

In [ ]:
r, p_value = stats.pearsonr(summary_stats_main['freezing_norm_dlc'],
               summary_stats_main['freezing_norm_rt'],)

print('Correlation coefficient between Deeplabcut vs Rodrigo')
print(f"Pearson's correlation coefficient is {round(r, 2)} with an associated p value of {p_value}")

In [ ]:
r, p_value = stats.pearsonr(summary_stats_main['freezing_norm_dlc'],
               summary_stats_main['freezing_norm_ez'],)

print('Correlation coefficient between Deeplabcut vs EzTrack (from Cai Lab)')
print(f"Pearson's correlation coefficient is {round(r, 2)} with an associated p value of {p_value}")

### 3.3) Linear Regressions

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Data to use in the regressions
freezing_rt = summary_stats_main[['freezing_norm_rt']]
freezing_dlc = summary_stats_main['freezing_norm_dlc']
freezing_ez = summary_stats_main[['freezing_norm_ez']]

#### 3.3.1) Rodrigo vs Deeplabcut

In [ ]:
lm = LinearRegression()
lm.fit(freezing_rt, freezing_dlc)

In [ ]:
print('Rodrigo vs Deeplabcut')
print(f'f(x) = {lm.coef_}x + {lm.intercept_}')
print(f'Coefficient of Determination : {round(lm.score(freezing_rt, freezing_dlc), 2)}')

#### 3.3.1) Rodrigo vs EZtrack

In [ ]:
lm = LinearRegression()
lm.fit(freezing_rt, freezing_ez)

In [ ]:
print('Rodrigo vs EZ Track')
print(f'f(x) = {(lm.coef_)}x + {lm.intercept_}')
print(f'Coefficient of Determination : {round(lm.score(freezing_rt, freezing_ez), 2)}')

# 4) Descriptive Statistics

In [ ]:
# Transform data into the long (ie "tidy" format)
summary_stats_main_long = summary_stats_main.melt(
    id_vars=['animal_id', 'cs_id', 'cs_epoch'],
    value_vars=['freezing_norm_dlc', 'freezing_norm_ez', 'freezing_norm_rt'],
    var_name='scorer',
    value_name='freezing_norm',
)

# Replace the name of scorers with shorter versions
summary_stats_main_long = summary_stats_main_long.replace(
    'freezing_norm_dlc', 'dlc').replace('freezing_norm_ez', 'ez').replace('freezing_norm_rt', 'rt')

summary_stats_main_long.head()

In [ ]:
summary_stats_main_long.head()

### 4.2 Distributions of freezing accross scorers

In [ ]:
sns.catplot(
    x='scorer',
    y='freezing_norm',
    data=summary_stats_main_long,
    hue='cs_id',
    col='cs_id',
    kind='box'
)

### 4.3) What about the general distribution?

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

sns.distplot(
    summary_stats_main_long[summary_stats_main_long['scorer']=='dlc'].freezing_norm,
    ax=axes[0], 
    bins= np.linspace(0, 1, 11)
)
sns.distplot(
    summary_stats_main_long[summary_stats_main_long['scorer']=='rt'].freezing_norm,
    ax=axes[1],
    bins= np.linspace(0, 1, 11),
    color='orange'
)
sns.distplot(
    summary_stats_main_long[summary_stats_main_long['scorer']=='ez'].freezing_norm,
    ax=axes[2],
    bins= np.linspace(0, 1, 11),
    color='grey'
)
for index, scorer in enumerate(['dlc', 'rt', 'ez']):
    axes[index].set(title=f'Freezing scored by: {scorer}')

### 4.4) Magnitude of freezing accress scorers, accross cs

In [ ]:
sns.relplot(
    x='scorer',
    y='freezing_norm',
    data=summary_stats_main_long,
    hue='cs_id',
    col='cs_id',
    kind='line',
)
plt.ylim(0, 1)

In [ ]:
sns.catplot(
    x='scorer',
    y='freezing_norm',
    data=summary_stats_main_long,
    hue='cs_id',
    estimator=np.mean,
    kind='bar',
)
plt.ylim(0, 1)

### 4.5) Overall freezing

In [ ]:
freezing = summary_stats_main_long.pivot_table(
    values=['freezing_norm'],
    index=['animal_id'],
    columns=['cs_id', 'scorer'],
)

freezing = freezing.groupby(axis=1, level=2).mean()
freezing.head()

In [ ]:
fig, ax = plt.subplots(figsize=(3, 5))
ax.bar(
    x=freezing.columns,
    height=freezing.mean(), 
    yerr=freezing.sem()
)

ax.set(ylim=(0, 1), title='freezing normalized', ylabel='freezing_norm', xlabel='scorer')

### 4.6) Are means significantly different?

In [ ]:
freezing.reset_index(inplace=True)

In [ ]:
# Preparing the data
freezing_long = freezing.melt(
    id_vars='animal_id',
    value_vars=['dlc', 'ez', 'rt'],
    var_name='scorer',
    value_name='freezing_norm'
)

In [ ]:
pg.kruskal(
    freezing_long,
    dv='freezing_norm', 
    between='scorer',
    detailed=True
)

In [ ]:
stats.kruskal(freezing.rt, freezing.dlc, freezing.ez)